In [1]:
import torch
import multiModel as mm
from multiModel import MultiInputModel
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
#Zapisz wyniki w 
path_results = "training_results_80_224_2"
# Utwórz katalog docelowy, jeśli nie istnieje
if not os.path.exists(path_results):
    print(f"Katalog '{path_results}' utworzony.")
    # Tworzenie katalogu, jeśli nie istnieje
    os.makedirs(path_results, exist_ok=True)

# Włącz blokowanie błędów CUDA
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Załaduj dane
train_dataset = mm.MultiInputDataset("CSV/dataset/train_80.csv", transform_rgb=mm.transform_rgb_224, transform_binary=mm.transform_binary_224)
val_dataset = mm.MultiInputDataset("CSV/dataset/val_80.csv", transform_rgb=mm.transform_rgb_224, transform_binary=mm.transform_binary_224)
test_dataset = mm.MultiInputDataset("CSV/dataset/test_80.csv", transform_rgb=mm.transform_rgb_224, transform_binary=mm.transform_binary_224)


# Inicjalizacja modelu
model = mm.CustomMultiInputModel(num_classes=11)
model = model.to("cuda")  # Jeśli masz GPU

#Dynamicznie przydzielany batch_size

batch_size = 64
model_name = "mm" + str(batch_size)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Optymalizator i funkcja straty
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Zapis logów
log_file = f"{path_results}/training_log_{model_name}.txt"
with open(log_file, "w") as f:
    f.write("epoch,train_loss,val_loss,train_accuracy,val_accuracy,train_precision,val_precision,train_recall,val_recall,train_f1,val_f1\n")

# Wczesne zatrzymanie - parametry
early_stop_patience = 5  # Liczba epok bez poprawy
best_val_loss = float("inf")
patience_counter = 0
best_model_path = f"{path_results}/best_model_{model_name}.pth"

# Pętla treningowa
num_epochs = 50
for epoch in range(num_epochs):
    # === TRENING ===
    print(f"Epoch {epoch + 1}/{num_epochs}")
    model.train()
    train_loss = 0
    train_true = []
    train_pred = []

    # Dodaj pasek postępu do pętli batchy
    with tqdm(total=len(train_loader), desc="Training", unit="batch") as pbar:
        for t_image, b_image, s_image, labels in train_loader:
            t_image, b_image, s_image, labels = (
                t_image.to("cuda"),
                b_image.to("cuda"),
                s_image.to("cuda"),
                labels.to("cuda")
            )

            # Oblicz predykcje i stratę
            outputs = model(t_image, b_image, s_image)
            loss = criterion(outputs, labels)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            train_true.extend(labels.cpu().numpy())
            train_pred.extend(predicted.cpu().numpy())
            pbar.set_postfix({"loss": f"{train_loss / (pbar.n + 1):.4f}"})  # Wyświetl średnią stratę
            pbar.update(1)  # Aktualizuj pasek postępu o 1 krok

    train_loss /= len(train_loader)  # Średnia strata w treningu
    train_accuracy = accuracy_score(train_true, train_pred)
    train_precision = precision_score(train_true, train_pred, average="weighted")
    train_recall = recall_score(train_true, train_pred, average="weighted")
    train_f1 = f1_score(train_true, train_pred, average="weighted")
    #print(f"Train Loss: {train_loss:.4f}")

    # === WALIDACJA ===
    model.eval()
    val_loss = 0
    val_true = []
    val_pred = []

    with torch.no_grad():  # Wyłącz gradienty
        with tqdm(total=len(val_loader), desc="Validation", unit="batch") as pbar_val:
            for t_image, b_image, s_image, labels in val_loader:
                t_image, b_image, s_image, labels = (
                    t_image.to("cuda"),
                    b_image.to("cuda"),
                    s_image.to("cuda"),
                    labels.to("cuda")
                )
                outputs = model(t_image, b_image, s_image)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                val_true.extend(labels.cpu().numpy())
                val_pred.extend(predicted.cpu().numpy())

                pbar_val.update(1)  # Aktualizuj pasek postępu walidacji

    val_loss /= len(val_loader)  # Średnia strata w walidacji
    val_accuracy = accuracy_score(val_true, val_pred)
    val_precision = precision_score(val_true, val_pred, average="weighted")
    val_recall = recall_score(val_true, val_pred, average="weighted")
    val_f1 = f1_score(val_true, val_pred, average="weighted")
    #print(f"Val Loss: {val_loss:.4f}")

    # === LOGI ===
    #print(f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
    with open(log_file, "a") as f:
        f.write(
            f"{epoch + 1},{train_loss:.4f},{val_loss:.4f},{train_accuracy:.4f},{val_accuracy:.4f},"
            f"{train_precision:.4f},{val_precision:.4f},{train_recall:.4f},{val_recall:.4f},{train_f1:.4f},{val_f1:.4f}\n"
        )

    # === WCZESNE ZATRZYMANIE ===
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # Zapis najlepszego modelu
        torch.save(model, best_model_path) #Zapisanie modelu i architektury w pliku pth
        print(f"Best model saved at epoch {epoch + 1}")
    else:
        patience_counter += 1
        print(f"No improvement in val loss for {patience_counter} epoch(s)")

    if patience_counter >= early_stop_patience:
        print("Early stopping triggered. Training stopped.")
        break
    


Katalog 'training_results_80_224_2' utworzony.
Epoch 1/50


Validation: 100%|██████████| 233/233 [00:49<00:00,  4.74batch/s]


Best model saved at epoch 1
Epoch 2/50


Validation: 100%|██████████| 233/233 [00:42<00:00,  5.43batch/s]


Best model saved at epoch 2
Epoch 3/50


...

No improvement in val loss for 3 epoch(s)
Epoch 36/50


Validation: 100%|██████████| 233/233 [00:41<00:00,  5.64batch/s]


No improvement in val loss for 4 epoch(s)
Epoch 37/50


Validation: 100%|██████████| 233/233 [00:41<00:00,  5.56batch/s]

No improvement in val loss for 5 epoch(s)
Early stopping triggered. Training stopped.


In [2]:
import torch
from multiModel import MultiInputModel, MultiInputDataset
import multiModel as mm
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from tqdm import tqdm

# Set CUDA error blocking
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Paths and parameters
path_results = "training_results_80_224_2"
model_name = "mm64"
best_model_path = f"{path_results}/best_model_{model_name}.pth"
test_csv = "CSV/dataset/test_80.csv"
batch_size = 64
num_classes = 11
class_names = [str(i) for i in range(num_classes)]  # Update with actual class names if available

# Create results directory if it doesn't exist
os.makedirs(path_results, exist_ok=True)

# Load test dataset
test_dataset = MultiInputDataset(test_csv, transform_rgb=mm.transform_rgb_224, transform_binary=mm.transform_binary_224)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load the trained model
model = torch.load(best_model_path)
model = model.to("cuda")
model.eval()

# Loss function
criterion = nn.CrossEntropyLoss()

# Test loop
test_loss = 0
test_true = []
test_pred = []

with torch.no_grad():
    with tqdm(total=len(test_loader), desc="Testing", unit="batch") as pbar:
        for t_image, b_image, s_image, labels in test_loader:
            t_image, b_image, s_image, labels = (
                t_image.to("cuda"),
                b_image.to("cuda"),
                s_image.to("cuda"),
                labels.to("cuda")
            )
            outputs = model(t_image, b_image, s_image)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            test_true.extend(labels.cpu().numpy())
            test_pred.extend(predicted.cpu().numpy())
            pbar.update(1)

# Calculate metrics
test_loss /= len(test_loader)
test_accuracy = accuracy_score(test_true, test_pred)
test_precision = precision_score(test_true, test_pred, average="weighted")
test_recall = recall_score(test_true, test_pred, average="weighted")
test_f1 = f1_score(test_true, test_pred, average="weighted")

# Log test metrics
test_log_file = f"{path_results}/test_results_{model_name}.txt"
with open(test_log_file, "w") as f:
    f.write("test_loss,test_accuracy,test_precision,test_recall,test_f1\n")
    f.write(f"{test_loss:.4f},{test_accuracy:.4f},{test_precision:.4f},{test_recall:.4f},{test_f1:.4f}\n")

# Compute confusion matrix
cm = confusion_matrix(test_true, test_pred)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()

# Save confusion matrix plot
cm_plot_path = f"{path_results}/confusion_matrix_{model_name}.png"
plt.savefig(cm_plot_path)
plt.close()

print(f"Test results saved to {test_log_file}")
print(f"Confusion matrix saved to {cm_plot_path}")
print(f"Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, "
      f"Recall: {test_recall:.4f}, F1: {test_f1:.4f}")

/tmp/ipykernel_55378/4230748760.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(best_model_path)
Testing: 100%|██████████| 233/233 [01:06<00:00,  3.5

Test results saved to training_results_80_224_2/test_results_mm64.txt
Confusion matrix saved to training_results_80_224_2/confusion_matrix_mm64.png
Test Loss: 0.2039, Accuracy: 0.9367, Precision: 0.9383, Recall: 0.9367, F1: 0.9366


In [1]:
import torch
import multiModel as mm
from multiModel import MultiInputModel
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

path_results = "training_results_80_224_2"
test_dataset = mm.MultiInputDataset("CSV/dataset/test_80.csv", transform_rgb=mm.transform_rgb_224, transform_binary=mm.transform_binary_224)
batch_size = 64
model_name = "mm" + str(batch_size)
best_model_path = f"{path_results}/best_model_{model_name}.pth"
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
# === TEST ===

# Inicjalizacja modelu
#model = mm.CustomMultiInputModel(num_classes=11)
model = torch.load(best_model_path)
#model = model.to("cuda")  # Jeśli masz GPU
# Wczytaj najlepszy model
#model= torch.load(best_model_path)
model.eval()

test_loss = 0
correct = 0
total = 0
with torch.no_grad():
    for t_image, b_image, s_image, labels in test_loader:
        t_image, b_image, s_image, labels = (
            t_image.to("cuda"),
            b_image.to("cuda"),
            s_image.to("cuda"),
            labels.to("cuda")
        )
        outputs = model(t_image, b_image, s_image)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Oblicz dokładność
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

test_loss /= len(test_loader)
accuracy = correct / total

# Zapis wyniku testu
log_file_test = f"training_results/test_log_{model_name}.txt"
with open(log_file_test, "w") as f_t:
        f_t.write(f"Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Zwalnianie pamięci po zakończeniu pracy z modelem
del model  # Usuń model z pamięci
torch.cuda.empty_cache()  # Wyczyść pamięć GPU

/tmp/ipykernel_48901/4079286112.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(best_model_path)


NameError: name 'criterion' is not defined